In [1]:
# 1. Imports
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import xml.etree.ElementTree as ET
import torch
import torch.nn as nn
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from datasets import Dataset
from accelerate import Accelerator

# Imports specific to the custom model
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType

# 2. Setup Environment Variables and Accelerator
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
accelerator = Accelerator()

In [2]:
# 3. Helper Functions
def convert_binding_string_to_labels(binding_string):
    """Convert 'proBnd' strings into label arrays."""
    return [1 if char == '+' else 0 for char in binding_string]

def truncate_labels(labels, max_length):
    """Truncate labels to the specified max_length."""
    return [label[:max_length] for label in labels]

def compute_metrics(p):
    """Compute metrics for evaluation."""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    predictions = predictions[labels != -100].flatten()
    labels = labels[labels != -100].flatten()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    auc = roc_auc_score(labels, predictions)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc}

def compute_loss(model, inputs):
    """Custom compute_loss function."""
    logits = model(**inputs).logits
    labels = inputs["labels"]
    loss_fct = nn.CrossEntropyLoss(weight=class_weights)
    active_loss = inputs["attention_mask"].view(-1) == 1
    active_logits = logits.view(-1, model.config.num_labels)
    active_labels = torch.where(
        active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
    )
    loss = loss_fct(active_logits, active_labels)
    return loss

In [3]:
from datasets import load_dataset

# データセットの読み込み
dataset = load_dataset("AmelieSchreiber/binding_sites_random_split_by_family")

FileNotFoundError: Couldn't find a dataset script at /work/AmelieSchreiber/binding_sites_random_split_by_family/binding_sites_random_split_by_family.py or any data file in the same directory. Couldn't find 'AmelieSchreiber/binding_sites_random_split_by_family' on the Hugging Face Hub either: FileNotFoundError: No (supported) data files or dataset script found in AmelieSchreiber/binding_sites_random_split_by_family. 

In [ ]:
dataset